In [273]:
import numpy as np
import pandas as pd

import requests
from lxml import html

pageContent=requests.get(
     'https://en.wikipedia.org/wiki/List_of_PlayStation_4_games'
)
tree = html.fromstring(pageContent.content)

counter = 3 

html_text = tree.xpath(
    f'/html/body/div[3]/div[3]/div[4]/div/table[2]//tr[{counter}]//text()')

assembled_list=[]

while html_text:
    cleaned_html_text=[]
    for i in range(len(html_text)-1):
        if html_text[i] != '\n' and html_text[i+1] != '\n':
            j=1
            while html_text[i+j] != '\n':
                html_text[i] = html_text[i] + html_text[i+j]
                html_text[i+j] = '\n'
                j += 1

    for i in html_text:
        if i != '\n':
            cleaned_html_text.append(i.replace('\n',''))
      
    assembled_list.append(cleaned_html_text[0:7])
    counter += 1
    html_text = tree.xpath(
    f'/html/body/div[3]/div[3]/div[4]/div/table[2]//tr[{counter}]//text()')

In [274]:
pageContent=requests.get(
     'https://en.wikipedia.org/wiki/List_of_PlayStation_4_games_(M-Z)'
)
tree = html.fromstring(pageContent.content)

counter = 3 

html_text = tree.xpath(
    f'/html/body/div[3]/div[3]/div[4]/div/table/tbody/tr[{counter}]//text()')

while html_text:
    cleaned_html_text=[]
    for i in range(len(html_text)-1):
        if html_text[i] != '\n' and html_text[i+1] != '\n':
            j=1
            while html_text[i+j] != '\n':
                html_text[i] = html_text[i] + html_text[i+j]
                html_text[i+j] = '\n'
                j += 1

    for i in html_text:
        if i != '\n':
            cleaned_html_text.append(i.replace('\n',''))
      
    assembled_list.append(cleaned_html_text[0:7])
    counter += 1
    html_text = tree.xpath(
    f'/html/body/div[3]/div[3]/div[4]/div/table/tbody/tr[{counter}]//text()')
    
    

In [324]:
game_list = pd.DataFrame(assembled_list,columns=['Titles','Genres','Developers','Publishers','ReleaseDate_JP','ReleaseDate_EU','ReleaseDate_NA'] )

release_list = ['ReleaseDate_JP','ReleaseDate_EU','ReleaseDate_NA']

for col in release_list:
    game_list[col]=pd.to_datetime(game_list[col],infer_datetime_format=True,errors='coerce')


In [326]:
game_list.tail(5)

,Titles,Genres,Developers,Publishers,ReleaseDate_JP,ReleaseDate_EU,ReleaseDate_NA
2308,Zombie Army Trilogy,Tactical shooter,Rebellion Developments,505 Games,2015-03-06,2015-03-06,2015-03-06
2309,Zombie Army 4: Dead War,Tactical shooter,Rebellion Developments,505 Games,2020-02-04,2020-02-04,2020-02-04
2310,Zombie Vikings,Action,Zoink Games,Zoink Games,2015-09-02,2015-09-01,2015-09-01
2311,Zone of the Enders: The 2nd Runner – Mars,Hack and slashthird-person shooter,CygamesKonami,Konami,2018-09-06,2018-09-06,2018-09-04
2312,Zotrix,Shoot 'em up,Zerobit Games,UFO Interactive Games,NaT,2016-01-08,2015-12-08
